# Variational approach to continuum thermodynamics

Gradient-driven minimization of the total energy potential which includes Maxwell viscoelasticity 

In [ ]:
%matplotlib widget

In [ ]:
import sympy as sp
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from bmcs_utils.api import Cymbol
sp.init_printing()

In [ ]:
# Let's now introduce the state variables of the viscoplastic model with isotropic hardening
# Define the state variables
eps = Cymbol(r'\varepsilon', codename='varepsilon')
eps_vp = Cymbol(r'\varepsilon_\mathrm{vp}', codename='varepsilon_vp')
z = Cymbol(r'z', codename='z') # is the isotropic hardening variable
# Define the material parameters
E = Cymbol('E', codename='E') # is the stiffness
K = Cymbol('K', codename='K') # is the bulk modulus
# The corresponding Helmholtz free energy then reads
Psi_vp_ = sp.Rational(1,2) * E * (eps - eps_vp)**2 + sp.Rational(1,2) * K * z**2
Psi_vp_

In [ ]:
# Let's define the conjugate stresses by differentiating the Helmholtz free energy with respect to the strain
sig_vp_ = -sp.diff(Psi_vp_, eps_vp)
Z_ = sp.diff(Psi_vp_, z)
sig_vp_, Z_

In [ ]:
# Let's now construct a dissipation potential for viscoplasticity based on the Perzyna model. 
# The Perzyna model is a simple viscoplastic model that is based on the concept of a yield surface.
# The yield surface is defined by a yield function, which is a function of the stress tensor and the hardening variable.
# It expresses the ratio between the value oof the yield function and the current strength of the material, multiplied by the fluidity parameter.
# The fluidity parameter is a material property that describes the rate at which the material flows under stress.
# Let's now define the symbols for material parameters, namely the yield stress, the hardening modulus, and the fluidity parameter.
sigma_y, theta = sp.symbols('sigma_y theta')
# Let us also define their rates as the time derivatives of the state variables.
dot_eps_vp = Cymbol(r'\dot{\varepsilon}_{vp}', codename='dot_eps_vp')
dot_z = Cymbol(r'\dot{z}', codename='dot_z')
# Considering just a scalar stress variable, let's now define the yield function, which is the stress minus the yield stress enhanced by the current value of hardening.
f_ = sp.simplify(sp.sqrt(sig_vp_**2) - (sigma_y + Z_))
f_

In [ ]:
# Now let's define an overstress of the viscoplastic model as the difference between the stress and the yield stress.
sig_over_ = sp.simplify(f_) #  / (sigma_y + Z_))
sig_over_

In [ ]:
# To construct the dissipation potential, we need to define the viscoplastic strain rate as the time derivative of the viscoplastic strain and of the hardening variable.
dt = Cymbol('\Delta t', codename='dt')
dot_eps_vp = Cymbol(r'\dot{\varepsilon}_\mathrm{vp}', codename='dot_eps_vp')
dot_z = Cymbol(r'\dot{z}', codename='dot_z')
# The dissipation potential is obtained by substituting the incremental representation of the viscoplastic strain and the hardening variable into the yield function.
eps_vp_1_ = eps_vp + dot_eps_vp * dt
z_1_ = z + dot_z * dt
sig_over_1_ = sig_over_.subs({eps_vp: eps_vp_1_, z: z_1_})
# The value of overstress in the trial state is multiplied with the fluidity parameter.
Phi_vp_ = theta * sp.Rational(1,2) * sp.Piecewise((sig_over_1_**2, f_ > 0), (0, True))
# However, this should be only positive, so we need to take the positive part of the expression.
Phi_vp_

In [ ]:
Pi_vp_ = Psi_vp_ + Phi_vp_
dPi_ddot_Eps_ = sp.Matrix([sp.diff(Pi_vp_, dot_eps_vp), sp.diff(Pi_vp_, dot_z)])
dPi_ddot_Eps_

In [ ]:
# Lambdify the functions
get_Pi_ = sp.lambdify((eps, eps_vp, z, dot_eps_vp, dot_z, dt, E, theta, sigma_y, K), Pi_vp_, 'numpy')

# Lambdify the derivatives
get_dPi_ddot_Eps_ = sp.lambdify((eps, eps_vp, z, dot_eps_vp, dot_z, dt, E, theta, sigma_y, K), dPi_ddot_Eps_, 'numpy')

In [ ]:
get_Pi_(0.1, 0, 0, 0, 0, 0.01, 30000, 1000, 100, 1000)

In [ ]:
# Time integrator for viscoelasticity
def time_integrator_vp(eps_t, E, theta, dt, sigma_y, K):
    n_steps = len(eps_t)
    eps_vp_t = np.zeros(n_steps)
    z_t = np.zeros(n_steps)
    sig_t = np.zeros(n_steps)
    dot_eps_vp_next = 0
    dot_z_next = 0
    for i in range(1, n_steps):
        eps_vp_prev = eps_vp_t[i-1]
        z_prev = z_t[i-1]
        
        def objective(x):
            dot_eps_vp, dot_z = x
            Pi_val = get_Pi_(eps_t[i], eps_vp_prev, z_prev, dot_eps_vp, dot_z, dt, E, theta, sigma_y, K)
            return Pi_val
        
        def gradient(x):
            dot_eps_vp, dot_z = x
            return np.array([get_dPi_ddot_Eps_(eps_t[i], eps_vp_prev, z_prev, dot_eps_vp, dot_z, dt, E, theta, sigma_y, K)])

        # res = minimize(objective, [dot_eps_vp_next, dot_z_next])

        res = minimize(objective, [dot_eps_vp_next, dot_z_next],
                       jac=gradient,
                       bounds=[(None, None),(0, None)], 
                       method='L-BFGS-B')
        
        dot_eps_vp_next, dot_z_next = res.x
        eps_vp_t[i] = eps_vp_prev + dot_eps_vp_next * dt
        z_t[i] = z_prev + dot_z_next * dt
        sig_t[i] = E * (eps_t[i] - eps_vp_t[i])
    
    return eps_t, eps_vp_t, sig_t, z_t

In [ ]:
# Example usage
E_val = 210e9  # Young's modulus in Pa
theta_val = 1e5  # Viscosity in Pa.s
K_val = 10e9  # Bulk modulus in Pa
sigma_y_val = 100e6  # Yield stress in Pa
dt = 0.01  # Time step in seconds
final_time = 10  # Final time in seconds
# Let's generate a cyclic sinusoidal loading history
time = np.arange(0, final_time, dt)
eps_t_2_cycles = 0.01 * np.sin(2 * np.pi * 2 * time / final_time)  # 2 cycles
eps_t_4_cycles = 0.01 * np.sin(2 * np.pi * 4 * time / final_time)  # 4 cycles
# Compute responses for both loading histories
eps_t_2, eps_vp_t_2, sig_t_2, z_t_2 = time_integrator_vp(eps_t_2_cycles, E_val, theta_val, dt, sigma_y_val, K_val)
eps_t_4, eps_vp_t_4, sig_t_4, z_t_4 = time_integrator_vp(eps_t_4_cycles, E_val, theta_val, dt, sigma_y_val, K_val)
# Elastic part
eps_e_t_2 = eps_t_2 - eps_vp_t_2
eps_e_t_4 = eps_t_4 - eps_vp_t_4
# Plot results
plt.figure(figsize=(8, 12))
plt.subplot(4, 1, 1)
plt.plot(time, eps_t_2_cycles, label='Total Strain (2 cycles)')
plt.plot(time, eps_e_t_2, label='Elastic Strain (2 cycles)')
plt.plot(time, eps_vp_t_2, label='Viscoplastic Strain (2 cycles)')
plt.plot(time, eps_e_t_2 + eps_vp_t_2, color='red', label='Sum e-vp (2 cycles)')
plt.plot(time, z_t_2, color='magenta', label='hardening variable')
plt.xlabel('Time [s]')
plt.ylabel('Strain')
plt.legend()
plt.title('Strain Components for 2 Cycles')
plt.subplot(4, 1, 2)
plt.plot(time, eps_t_4_cycles, label='Total Strain (4 cycles)')
plt.plot(time, eps_e_t_4, label='Elastic Strain (4 cycles)')
plt.plot(time, eps_vp_t_4, label='Viscoplastic Strain (4 cycles)')
plt.xlabel('Time [s]')
plt.ylabel('Strain')
plt.legend()
plt.title('Strain Components for 4 Cycles')
plt.subplot(4, 1, 3)
plt.plot(time, sig_t_2, label='Stress (2 cycles)')
plt.plot(time, sig_t_4, label='Stress (4 cycles)')
plt.xlabel('Time [s]')
plt.ylabel('Stress [Pa]')
plt.legend()
plt.title('Stress Response for Different Loading Rates')
plt.subplot(4, 1, 4)
plt.plot(eps_t_2_cycles, sig_t_2, label='Stress-strain (2 cycles)')
plt.plot(eps_t_4_cycles, sig_t_4, label='Stress-Strain (4 cycles)')
plt.xlabel('Strain [-]')
plt.ylabel('Stress [Pa]')
plt.legend()
plt.title('Stress Response for Different Loading Rates')
plt.tight_layout()
plt.show()